In [7]:
"""MEV Specification: NL from base model.

Swiss Metro Case Study
Demand Modeling: 1.202
Spring 2019
Alexandra Berke

"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models

pandas = pd.read_table("../Data/swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-20,20,0)
ASC_SBB	 = Beta('ASC_SBB',0,-10,10,1)
ASC_SM	 = Beta('ASC_SM',0,-20,20,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-10,10,0)
B_HE	 = Beta('B_HE',0,-10,10,0)
B_SM_COST	 = Beta('B_SM_COST',0,-10,10,0)
B_CAR_TIME	 = Beta('B_CAR_TIME',0,-10,10,0)
B_TRAIN_TIME	 = Beta('B_TRAIN_TIME',0,-10,10,0)
B_SM_TIME	 = Beta('B_SM_TIME',0,-10,10,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-10,10,0)
B_SENIOR	 = Beta('B_SENIOR',0,-10,10,0)
B_GA	 = Beta('B_GA',0,-10,80,0)
B_FIRST	 = Beta('B_FIRST',0,-10,80,0)
# parameters relevant to the nests
MU_rail = Beta('MU_rail',1,1,10,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data

SENIOR  = DefineVariable('SENIOR', AGE   ==  5,database )
CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

# Utilities
Car_SP = ASC_CAR * one + B_CAR_TIME * CAR_TT + B_CAR_COST * CAR_CO + B_SENIOR * SENIOR
SBB_SP = ASC_SBB * one + B_TRAIN_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE \
        + B_GA * GA + B_FIRST * FIRST
SM_SP = ASC_SM * one + B_SM_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE + B_GA * GA \
        + B_SENIOR * SENIOR + B_FIRST * FIRST

V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Definition of nests
car = 1.0, [3]
rail = MU_rail, [1, 2]
nests = car, rail

# Estimation of the model
logprob = models.lognested(V,av,nests,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MEV_SM_base_NL"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_CAR=	-0.252
ASC_SM=	0.0239
B_CAR_COST=	-0.00601
B_CAR_TIME=	-0.0111
B_FIRST=	0.244
B_GA=	0.64
B_HE=	-0.00156
B_SENIOR=	-0.49
B_SM_COST=	-0.0125
B_SM_TIME=	-0.00401
B_TRAIN_COST=	-0.0194
B_TRAIN_TIME=	-0.00287
MU_rail=	4.04
                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR      -0.252058  0.092640  -2.720835  6.511726e-03      0.101617   
ASC_SM        0.023876  0.027940   0.854566  3.927917e-01      0.027676   
B_CAR_COST   -0.006005  0.001058  -5.676659  1.373508e-08      0.001487   
B_CAR_TIME   -0.011067  0.000793 -13.948766  0.000000e+00      0.001588   
B_FIRST       0.244444  0.072227   3.384369  7.134214e-04      0.078525   
B_GA          0.640061  0.187348   3.416420  6.345031e-04      0.199964   
B_HE         -0.001563  0.000321  -4.865988  1.138865e-06      0.000404   
B_SENIOR     -0.489598  0.061336  -7.982256  1.332268e-15      0.097653   
B_SM_COST    -0.012451  0.000582 -21.375390  0.000000e+00      0.000925   
B_SM_TIME    -0.004014 